In [3]:
# 備考１(行番号)	備考２(顧客番号)	表示名	ﾌﾘｶﾞﾅ(顧客)	ﾌﾘｶﾞﾅ(会社)	会社名	ボトル1	ボトル1番号	ボトル2	ボトル2番号	ボトル3	ボトル3番号
# Remark 1 (line number) Remark 2 (customer number) Display name Furigana (customer)	Furigana (Company) Company name Bottle 1 Bottle 1 number Bottle 2 Bottle 2 number Bottle 3 Bottle 3 number

# 1	3	相田  得皓	ｱｲﾀﾞ　ﾄｸﾋﾛ	ｱｲﾀﾞｳﾝﾕ	相田運輸㈱
# 2	4	青木  道男	ｱｵｷ　ﾐﾁｵ	ｱｵｷﾓｸｻﾞｲ	青木木材㈱iiii
import pandas as pd
import numpy as np
import uuid
from datetime import datetime
import jaconv
import json
import time

In [4]:
str(uuid.uuid4())

'71daf450-03d2-4d18-ad13-729d6cb90a46'

In [5]:
dfs = pd.read_excel("data.xlsx", sheet_name=None)

In [6]:
sheet = dfs["Sheet1"]
cols = sheet.columns

In [7]:
bots1 = sheet["ボトル1"][sheet["ボトル1"].notnull()]
bots2 = sheet["ボトル2"][sheet["ボトル2"].notnull()]
bots3 = sheet["ボトル3"][sheet["ボトル3"].notnull()]
bottles = set(bots1.to_list() + bots2.to_list() + bots3.to_list())

In [8]:
now_date = datetime.utcnow().isoformat()[:-3]+ "Z"
time.sleep(5)

In [9]:

now_date2 = datetime.utcnow().isoformat()[:-3]+ "Z"

In [10]:
now_date2

'2022-04-26T11:17:03.803Z'

In [11]:
drinks = dict()
drinks_id = dict()
for bottle in bottles:
    uid = str(uuid.uuid4())
    drinks_id[bottle] = uid
    drinks[uid] = {
        "attributes": {
            "name": bottle,
            "url": ""
        },
        "createdAt": now_date,
        "updatedAt": now_date
    }
    

In [19]:
users = dict()
all_orders = dict()
for index, row in sheet.iterrows():
    na_row = row.isna()
#     my_bottles = []
    my_bottles = [(row[f"ボトル{i}"], str(row[f"ボトル{i}番号"])) for i in range(1,4) if not na_row[f"ボトル{i}"]]
    
    uid = str(uuid.uuid4())
    orders = dict()
    for my_bottle in my_bottles:
        uid_order = str(uuid.uuid4())
        
        alreadyExists = dict(filter(lambda o:  o[1]["attributes"]["drinkId"] == drinks_id[my_bottle[0]] and o[1]["attributes"]["drinkCode"] == my_bottle[1],all_orders.items()))
        if alreadyExists:
            uid_order = list(alreadyExists)[0]
        orders[uid_order] = {
            "drinkCode": my_bottle[1],
            "drinkId": drinks_id[my_bottle[0]]
        }
        all_orders[uid_order] = {
          "attributes": {
            "drinkCode": my_bottle[1],
            "drinkId": drinks_id[my_bottle[0]],
            "inUse": False,
            "userId": uid
          },
          "createdAt": now_date,
          "uid": uid_order,
          "updatedAt": now_date
        }
    users[uid] = {
      "attributes": {
        "birthday": "",
        "email": "" if na_row['電子メール アドレス'] else row['電子メール アドレス'],
        "furigana": jaconv.h2z("" if na_row['ﾌﾘｶﾞﾅ(顧客)'] else row['ﾌﾘｶﾞﾅ(顧客)']),
        "introducer": "",
        "isInStore": False,
        "job": "" if na_row['会社名'] else  row['会社名'],
        "memo": "",
        "name": "" if na_row['表示名'] else row['表示名'],
        "phone": "" if na_row['携帯電話'] else row['携帯電話'],
        "photo": "",
        "customerNumber": row['備考２(顧客番号)']
      },
      "createdAt": now_date,
      "updatedAt": now_date2,
      "relationships": {
        "orders": orders
      }
    }
#     if(index > 200): break

In [18]:
# na_row['備考２(顧客番号)']
na_row

備考１(行番号)      False
備考２(顧客番号)     False
表示名           False
ﾌﾘｶﾞﾅ(顧客)     False
氏名            False
役職名            True
ﾌﾘｶﾞﾅ(会社)     False
会社名           False
部署名           False
勤務先郵便番号       False
勤務先住所１        False
勤務先住所２        False
勤務先電話番号       False
勤務先ファックス       True
携帯電話          False
電子メール アドレス    False
ボトル1           True
ボトル1番号         True
ボトル2           True
ボトル2番号         True
ボトル3           True
ボトル3番号         True
Name: 11205, dtype: bool

In [15]:
with open ('bar.json', "r") as f:
    data_ = json.loads(f.read())

In [20]:
all_data = dict()
all_data["bottles"] = drinks
all_data["users"] = users
all_data["orders"] = all_orders
all_data["admins"] = data_["admins"]
all_data["roles"] = data_["roles"]
with open('new_data.json', 'w', encoding='utf8') as fp:
    json.dump(all_data, fp, ensure_ascii=False)

In [21]:
!open .